In [1]:
import json
import sqlite3
from datetime import datetime
from pathlib import Path

In [2]:
INPUT_FILE = Path("daily_summary.json")

if not INPUT_FILE.exists():
    raise FileNotFoundError("daily_summary.json not found")

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    sessions = json.load(f)

print("Loaded entries:", len(sessions))
sessions


Loaded entries: 2


[{'start': '12:01',
  'end': '12:02',
  'task': 'Tracking activity in Python Jupyter notebook'},
 {'start': '12:04',
  'end': '12:08',
  'task': 'Setting up Tesseract OCR software and reviewing documentation'}]

In [3]:
DB_FILE = Path("timesheet.db")

conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS timesheet (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,
    start TEXT,
    end TEXT,
    hours REAL,
    task TEXT
)
""")

conn.commit()

print("Connected →", DB_FILE.resolve())


Connected → C:\Users\GCV\dev\work\auto-timesheet-llm-poc\notebooks\timesheet.db


In [4]:
today = datetime.now().strftime("%Y-%m-%d")

inserted = 0

for s in sessions:
    try:
        start_dt = datetime.strptime(s["start"], "%H:%M")
        end_dt = datetime.strptime(s["end"], "%H:%M")

        duration_hours = (end_dt - start_dt).total_seconds() / 3600

        if duration_hours <= 0:
            continue

        cursor.execute("""
            INSERT INTO timesheet (date, start, end, hours, task)
            VALUES (?, ?, ?, ?, ?)
        """, (
            today,
            s["start"],
            s["end"],
            round(duration_hours, 2),
            s["task"]
        ))

        inserted += 1

    except Exception as e:
        print("Skipped entry:", s, "| Error:", e)

conn.commit()

print(f"Inserted {inserted} valid entries into DB")


Inserted 2 valid entries into DB


In [5]:
rows = cursor.execute(
    "SELECT date, start, end, hours, task FROM timesheet ORDER BY id DESC"
).fetchall()

for r in rows:
    print(r)


('2026-02-17', '12:04', '12:08', 0.07, 'Setting up Tesseract OCR software and reviewing documentation')
('2026-02-17', '12:01', '12:02', 0.02, 'Tracking activity in Python Jupyter notebook')
